In [1]:
import numpy as np
import matplotlib.pyplot as plt
# from ROI_extraction import preprocess_image
import sys
sys.path.append("..")
from common_functions import HSV_features_generation

In [2]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_text
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
import xgboost as xgb

def eval_XGBoost(X,y,k,max_depth,num_rounds,learning_rate=0.3, min_split_loss=0,min_child_weight=1,max_delta_step=0,subsample=1,show_tree=False):
    # Initialize a list to store the accuracy scores
    accuracy_scores = []
    val_accuracy_scores = []
    test_accuracy_scores = []
    features_weights = []
    confusion_matrixs = []
    val_confusion_matrixs = []
    test_confusion_matrixs = []
    
    # Nombre de plis pour la validation croisée k-fold
    k = 4

    # Créer une instance de StratifiedKFold avec k plis
    skf = StratifiedKFold(n_splits=k)

    for i, (train_index, val_test_index) in enumerate(skf.split(X, y)):
        if(show_tree):
            print(f"Fold {i}:")
        # Spécifie les données d'entrainement
        X_train = X.iloc[train_index]
        y_train = y[train_index]

        # Spécifie les données de validation et de test
        skf2 = StratifiedKFold(n_splits=2)
        test_index,val_index = next(skf2.split(X.iloc[val_test_index], y[val_test_index]))
        X_val = X.iloc[val_test_index].iloc[val_index]
        y_val = y[val_test_index][val_index]
        X_test = X.iloc[val_test_index].iloc[test_index]
        y_test = y[val_test_index][test_index]

        # Créer l'objet DMatrix pour les données d'entraînement et de test
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)
        dtest = xgb.DMatrix(X_test, label=y_test)
        
        # Définir les paramètres du modèle XGBoost
        params = {
            'objective': 'multi:softmax',  # Fonction objective pour la classification binaire
            'max_depth': max_depth,  # Profondeur maximale de chaque arbre
            'num_class': 3,
            'learning_rate': learning_rate,
            'min_split_loss' : min_split_loss,
            'min_child_weight': min_child_weight,
            'max_delta_step':max_delta_step,
            'subsample':subsample
        }
        
        model = xgb.train(params, dtrain, num_rounds)

        # Prédiction sur les données de test
        y_train_pred = model.predict(dtrain)
        y_val_pred = model.predict(dval)
        y_test_pred = model.predict(dtest)

        # Calculate the accuracy score
        accuracy = metrics.accuracy_score(y_train, y_train_pred)
        val_accuracy = metrics.accuracy_score(y_val, y_val_pred)
        test_accuracy = metrics.accuracy_score(y_test, y_test_pred)

        confusion_mat = confusion_matrix(y_train, y_train_pred)
        confusion_mat_val = confusion_matrix(y_val, y_val_pred)
        confusion_mat_test = confusion_matrix(y_test, y_test_pred)

        if(show_tree):
            print(f'Fold {i} accuracy : {accuracy}, val accuracy :{val_accuracy}, test accuracy : {test_accuracy}')

        # Append the accuracy score to the list
        accuracy_scores.append(accuracy)
        val_accuracy_scores.append(val_accuracy)
        test_accuracy_scores.append(test_accuracy)
        confusion_matrixs.append(confusion_mat)
        val_confusion_matrixs.append(confusion_mat_val)
        test_confusion_matrixs.append(confusion_mat_test)

    # Compute the average accuracy across all folds
    if(show_tree):
        print('-------------------')
        print(f"Train accuracy : {np.mean(accuracy_scores)}")
        print(f"Val accuracy : {np.mean(val_accuracy_scores)}")

        fig, ax = plt.subplots(figsize=(4, 3))
        ax.set_xlabel('Prédiction')
        ax.set_ylabel('Vraie étiquette')
        ax.set_title('Matrice de confusion')
        print('Average confusion matrix')
        sns.heatmap(np.mean(confusion_matrixs, axis=0), annot=True, cmap="Blues", ax=ax, fmt='.0f')
        plt.show()

        fig, ax = plt.subplots(figsize=(4, 3))
        ax.set_xlabel('Prédiction')
        ax.set_ylabel('Vraie étiquette')
        ax.set_title('Matrice de confusion val')
        print('Average val confusion matrix')
        sns.heatmap(np.mean(val_confusion_matrixs, axis=0), annot=True, cmap="Blues", ax=ax, fmt='.0f')
        plt.show()

    return np.mean(accuracy_scores),np.mean(val_accuracy_scores),np.mean(test_accuracy_scores)



In [3]:
#raw
X,y = HSV_features_generation('../images/raw_pollub/')


In [4]:
eval_XGBoost(X,y,5,100,6,50,learning_rate=0.3, min_split_loss=0,min_child_weight=1,max_delta_step=0,subsample=1,show_tree=False)

[09:57:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[09:57:08] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[09:57:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[09:57:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimators" } are not used.



(1.0, 0.962097008422887, 0.9560702875399361)

In [ ]:
#ROI
X,y = HSV_features_generation('../images/ROI_pollub/')